In [1]:
import numpy as np
import matplotlib.pyplot as plt
#!pip install scipy
from scipy import stats
from scipy import optimize
from math import sin

In [2]:
class Auction:
    def __init__(self, *args, **kwargs):
        pass

    def get_winners(self, bids):
        pass

    def get_payments_per_click(self, winners, values, bids):
        pass

    def round(self, bids):
        winners, values = self.get_winners(bids) # allocation mechanism!
        payments_per_click = self.get_payments_per_click(winners, values, bids)
        return winners, payments_per_click

In [3]:
class FirstPriceAuction(Auction):
    def __init__(self, ctrs):
        self.ctrs = ctrs
        self.n_adv = len(self.ctrs)

    def get_winners(self, bids):
        adv_values = self.ctrs*bids
        adv_ranking = np.argsort(adv_values)
        winner = adv_ranking[-1]
        return winner, adv_values

    def get_payments_per_click(self, winners, values, bids):
        payment = bids[winners]
        return payment.round(2)

Let's configure the parameters to have the same values of the other parts of the project

In [ ]:
import numpy as np

# Configuration parameters
ITERATIONS = 1000
AUCTIONS = 100
N_USERS = 100
N_ADVERTISERS = 10    # number of companies that compete for slot auctions, including myself
NUMBER_OF_ARMS = 100    # needed for the UCB algorithm
PRICES = np.linspace(0, 1, NUMBER_OF_ARMS)    # actual arms of the ucb algorithm
BUGDET = 1000 # I know it's mispelled but i found it like this in the other file so i kept it like this
NUMBER_OF_SLOTS = 10

lambda_fun_param = lambda s: 1/(s+(0.3))       # probability of the ad being seen given the position s ->  0.5 for first position, 0.33 for second, 0.25 for third, etc...
LAMBDAS = np.array([lambda_fun_param(i) for i in range(1, NUMBER_OF_SLOTS+1)])    # array of lambda values (for each slot)

Pattern definition

In [ ]:
### Adversarial Bidders
#### interest !!
my_valuation = 0.6 #used in c
rho = BUGDET/N_USERS

# non-trivial scenario: competitors sample bids from a uniform with range varying over time
pattern = lambda t: 1-np.abs(np.sin(5*t/N_USERS))
other_bids = np.array([np.random.uniform(0, pattern(t), size = N_ADVERTISERS) for t in range(N_USERS)]).T
# I assume that competitors may have a larger budget than mine, but they may
# not deplete it.

m_t = other_bids.max(axis=0)

plt.plot(m_t)
plt.title('Expected maximum bid')
plt.xlabel('$t$')
plt.ylabel('$m_t$')
plt.show()


clairvoyant

In [18]:
available_bids = np.linspace(0,1,11)
win_probabilities = np.array([sum(b > m_t)/N_USERS for b in available_bids])
## Linear Program
#linprog is a wayto optimize the function and the bid
c = -(my_valuation-available_bids)*win_probabilities
A_ub = [available_bids*win_probabilities]
b_ub = [rho]
A_eq = [np.ones(len(available_bids))]
b_eq = [1]
res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0,1))
gamma = res.x
expected_clairvoyant_utilities = [-res.fun for u in range(N_USERS)]
expected_clairvoyant_bids = [sum(available_bids*gamma*win_probabilities) for u in range(N_USERS)]

### Generalized Multiplicative Pacing for Non-Truthful Auctions

#### Here, we will assume a **full-feedback** setting. In auctions, full-feedback is equivalent of knowing the maximum bid $m_t$ that won the auction (of course, **after** the auction). Moreover, bids will be **discretized** into a finite set of actions.
Full feedback = You know th bid of each competitor

#### We will leverage a full-feedback, adversarial learner such as Hedge (see Lab 2).

In [10]:
class HedgeAgent:
    def __init__(self, narms, learning_rate): #ho tolto il numero di slot da questa classe per tenere solo narms
        self.narms = narms   #number of possible bids
        self.learning_rate = learning_rate #supposing it's the rate at which the algorithm progress
        self.weights = np.ones(narms) #weight == proportionnal to the probabilty
        self.x_t = np.ones(narms)/narms #probability of a bid to be played
        self.a_t = None # bid choice done during the round
        self.t = 0

    def pull_arm(self): #choose which arm to play
        self.x_t = self.weights/sum(self.weights)
        self.a_t = np.random.choice(np.arange(self.narms), p=self.x_t)
        return self.a_t

    def update(self, l_t): #update the weights
        self.weights *= np.exp(-self.learning_rate*l_t)
        self.t += 1

In [11]:
class FFMultiplicativePacingAgent:
    def __init__(self, bids_set, valuation, budget, T, eta):
        self.bids_set = bids_set #available choices
        self.narms = len(bids_set) #nuber of bid choices
        self.hedge = HedgeAgent(self.narms, np.sqrt(np.log(self.narms)/T)) #qui ci va narms o nslots?? ma come learning rate non ci andrebbe eta??
        self.valuation = valuation #what you earn, ctr*profit_per_click
        self.budget = budget
        self.eta = eta #learning rate
        self.T = T  #number of rounds
        self.rho = self.budget/self.T #bid limit
        self.lmbd = 1 #pacing multiplier, when we spend more than rho it increses otherwise we decrease it
        self.t = 0

    def bid(self):
        if self.budget < 1:
            return 0
        return self.bids_set[self.hedge.pull_arm()]

    def update(self, f_t, c_t, m_t):
        # update hedge
        f_t_full = np.array([(self.valuation-b)*int(b >= m_t) for b in self.bids_set])
        c_t_full = np.array([b*int(b >= m_t) for b in self.bids_set])
        L = f_t_full - self.lmbd*(c_t_full-self.rho)
        range_L = 2+(1-self.rho)/self.rho
        self.hedge.update((2-L)/range_L) # hedge needs losses in [0,1]
        # update lagrangian multiplier
        self.lmbd = np.clip(self.lmbd-self.eta*(self.rho-c_t),
                            a_min=0, a_max=1/self.rho)
        # update budget
        self.budget -= c_t

## Trial

First iteration

In [ ]:
eta = 1/np.sqrt(N_USERS)  # Learning rate from theory

# Initializing the agent and auction
agent = FFMultiplicativePacingAgent(bids_set=available_bids,
                                    valuation=my_valuation,
                                    budget=BUGDET,
                                    T=N_USERS,
                                    eta=eta)

auction = FirstPriceAuction(np.ones(N_ADVERTISERS + 1))  # Auction setup

# Arrays to store results
utilities = np.array([])  # Utility gained in each auction
my_bids = np.array([])  # Bids placed by the agent
my_payments = np.array([])  # Payments made by the agent
total_wins = 0  # Counter for total wins

# Running the auction process
np.random.seed(1)  # Seed for reproducibility

for u in range(N_USERS):
    # Agent places a bid
    my_bid = agent.bid()

    # Gather all bids for this auction
    bids = np.append(my_bid, other_bids[:, u].ravel())
    m_t_new = np.max(bids)

    # Determine winners and payments
    winners, payments_per_click = auction.round(bids=bids)

    # Check if the agent won
    my_win = int(winners == 0)  # Agent wins if winners is 0

    # Calculate the utility and payment for the agent
    f_t = (my_valuation - m_t_new) * my_win  # Utility when the agent wins
    c_t = m_t_new * my_win  # Payment if the agent wins

    # Update the agent's learning based on the auction outcome
    agent.update(f_t, c_t, m_t_new)

    # Store the results
    utilities = np.append(utilities, f_t)  # Append utility
    my_bids = np.append(my_bids, my_bid)  # Append bid
    my_payments = np.append(my_payments, c_t)  # Append payment
    total_wins += my_win  # Increment total wins

    cumulative_payments = np.cumsum(my_payments)

    # Nicely formatted output to track progress
    print(f"\nIteration {u}:")  # New line for each iteration
    print(f"  My bid            : {my_bid:.2f}")
    print(f"  All bids          : {bids}")
    print(f"  Winners           : {winners}")
    print(f"  My win            : {my_win}")
    print(f"  m_t[{u}]          : {m_t_new:.2f}")
    print(f"  c_t               : {c_t:.2f}")
    print(f"  Total wins so far : {total_wins}")
    print(f"  Current payment : {my_payments[u]}")
    print(f"  Total paid so far : {cumulative_payments[u]}")

# Display total number of wins at the end
print(f"Total number of wins: {total_wins}")


graphical representation

In [ ]:
plt.plot(my_bids)
plt.xlabel('$t$')
plt.ylabel('$b_t$')
plt.title('Chosen Bids')
plt.show()

In [ ]:
cumulative_payments = np.cumsum(my_payments)
plt.plot(cumulative_payments)
plt.xlabel('$t$')
plt.ylabel('$\sum c_t$')
plt.axhline(BUGDET, color='red', label='Budget')
plt.legend()
plt.title('Cumulative Payments of Multiplicative Pacing')
plt.show()

## ATTENTION: The payments line looks fixed to 0 but it's not, it reaches something like 22 (we are watching just the first iteration out of 1000) but since the budget is 1000 it looks flat

In [ ]:
cumulative_regret = np.cumsum(expected_clairvoyant_utilities-utilities)
plt.plot(cumulative_regret)
plt.xlabel('$t$')
plt.ylabel('$\sum R_t$')
plt.title('Cumulative Regret of Multiplicative Pacing')
plt.show()

## ATTENTION: The negative regret problem looks fixed

## now doing multiple rounds to have a better idea

In [25]:
all_regrets = []
all_payments = []
for i in range(ITERATIONS):
    agent = FFMultiplicativePacingAgent(bids_set = available_bids,
                                        valuation=my_valuation,
                                        budget=BUGDET,
                                        T=N_USERS,
                                        eta=eta)

    auction = FirstPriceAuction(np.ones(N_ADVERTISERS+1))
    utilities = np.array([])
    my_bids = np.array([])
    my_payments = np.array([])

    np.random.seed(i)
    for u in range(N_USERS):
        # interaction
        my_bid = agent.bid()
        bids = np.append(my_bid, other_bids[:, u].ravel())
        winners, payments_per_click = auction.round(bids=bids)
        my_win = int(winners==0)
        f_t, c_t = (my_valuation-m_t[u])*my_win, m_t[u]*my_win
        agent.update(f_t, c_t, m_t[u])
        # logging
        utilities = np.append(utilities, f_t)
        my_payments = np.append(my_payments, c_t)
    all_regrets.append(np.cumsum(expected_clairvoyant_utilities-utilities))
    all_payments.append(np.cumsum(my_payments))

avg_regret = np.array(all_regrets).mean(axis=0)
std_regret = np.array(all_regrets).std(axis=0)

avg_payments = np.array(all_payments).mean(axis=0)
std_payments = np.array(all_payments).std(axis=0)

graphical representation

In [ ]:
plt.plot(np.arange(n_users), avg_payments)
plt.fill_between(np.arange(n_users), avg_payments-std_payments, avg_payments+std_payments, alpha=0.3)
plt.xlabel('$t$')
plt.ylabel('$\sum c_t$')
plt.axhline(B, color='red', label='Budget')
plt.legend()
plt.title('Cumulative Payments of Multiplicative Pacing')
plt.show()

## ATTENTION: I think this graph shows the average amount paid in 1 iteration compared to the total budget, so it looks very similar to the single iteration graph, maybe we should change it and show the sum instead of the avg so we can see among the 1000 iterations which is the cumulative expense compared to the budget

In [ ]:
plt.plot(np.arange(N_USERS), avg_regret)
plt.fill_between(np.arange(N_USERS), avg_regret-std_regret, avg_regret+std_regret, alpha=0.3)
plt.xlabel('$t$')
plt.ylabel('$\sum R_t$')
plt.title('Cumulative Regret of Full-Feedback Multiplicative Pacing')
plt.show()

## ATTENTION: same reasoning that I did for the payments graph, maybe we should sow the total regret instead of the avg